# Deep cortical machine learning: Policy gradients + Q-learning + GAN


In this notebook, we'll build a neural network that can learn to play games through reinforcement learning. More specifically, we'll use Q-learning to train an agent to play a game called [Cart-Pole](https://gym.openai.com/envs/CartPole-v0). In this game, a freely swinging pole is attached to a cart. The cart can move to the left and right, and the goal is to keep the pole upright as long as possible.

![Cart-Pole](assets/cart-pole.jpg)

We can simulate this game using [OpenAI Gym](https://gym.openai.com/). First, let's check out how OpenAI Gym works. Then, we'll get into training an agent to play the Cart-Pole game.

In [1]:
# In this one we should define and detect GPUs for tensorflow
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


>**Note:** Make sure you have OpenAI Gym cloned into the same directory with this notebook. I've included `gym` as a submodule, so you can run `git submodule --init --recursive` to pull the contents into the `gym` repo.

##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import gym

## Create the Cart-Pole game environment
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('MountainCarContinuous-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


We interact with the simulation through `env`. To show the simulation running, you can use `env.render()` to render one frame. Passing in an action as an integer to `env.step` will generate the next step in the simulation.  You can see how many actions are possible from `env.action_space` and to get a random action you can use `env.action_space.sample()`. This is general to all Gym games. In the Cart-Pole game, there are two possible actions, moving the cart left or right. So there are two actions we can take, encoded as 0 and 1.

Run the code below to watch the simulation run.

In [3]:
env.reset()
batch = []
for _ in range(1111):
    #env.render()
    action = env.action_space.sample()
    state, reward, done, info = env.step(action) # take a random action
    batch.append([action, state, reward, done, info])
    #print('state, action, reward, done, info:', state, action, reward, done, info)
    if done:
        env.reset()

To shut the window showing the simulation, use `env.close()`.

In [4]:
# env.close()

If you ran the simulation above, we can look at the rewards:

In [5]:
batch[0], 
batch[0][1].shape, state.shape

((4,), (4,))

In [6]:
import numpy as np
actions = np.array([each[0] for each in batch])
states = np.array([each[1] for each in batch])
rewards = np.array([each[2] for each in batch])
dones = np.array([each[3] for each in batch])
infos = np.array([each[4] for each in batch])

In [7]:
# print(rewards[-20:])
print('shapes:', np.array(rewards).shape, np.array(states).shape, np.array(actions).shape, np.array(dones).shape)
print('dtypes:', np.array(rewards).dtype, np.array(states).dtype, np.array(actions).dtype, np.array(dones).dtype)
print('states:', np.max(np.array(states)), np.min(np.array(states)))
print('actions:', np.max(np.array(actions)), np.min(np.array(actions)))
# print((np.max(np.array(actions)) - np.min(np.array(actions)))+1)
print('rewards:', np.max(np.array(rewards)), np.min(np.array(rewards)))

shapes: (1111,) (1111, 4) (1111,) (1111,)
dtypes: float64 float64 int64 bool
states: 2.8460571322144843 -2.7299966635595387
actions: 1 0
rewards: 1.0 1.0


In [8]:
actions[:10]

array([0, 1, 1, 0, 1, 1, 1, 1, 1, 1])

In [9]:
rewards[:10]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [10]:
# import numpy as np
def sigmoid(x, derivative=False):
  return x*(1-x) if derivative else 1/(1+np.exp(-x))

In [11]:
sigmoid(np.max(np.array(rewards))), sigmoid(np.min(np.array(rewards)))

(0.7310585786300049, 0.7310585786300049)

In [12]:
print('rewards:', np.max(np.array(rewards))/100, np.min(np.array(rewards))/100)

rewards: 0.01 0.01


The game resets after the pole has fallen past a certain angle. For each frame while the simulation is running, it returns a reward of 1.0. The longer the game runs, the more reward we get. Then, our network's goal is to maximize the reward by keeping the pole vertical. It will do this by moving the cart to the left and the right.

## Q-Network

We train our Q-learning agent using the Bellman Equation:

$$
Q(s, a) = r + \gamma \max{Q(s', a')}
$$

where $s$ is a state, $a$ is an action, and $s'$ is the next state from state $s$ and action $a$.

Before we used this equation to learn values for a Q-_table_. However, for this game there are a huge number of states available. The state has four values: the position and velocity of the cart, and the position and velocity of the pole. These are all real-valued numbers, so ignoring floating point precisions, you practically have infinite states. Instead of using a table then, we'll replace it with a neural network that will approximate the Q-table lookup function.

<img src="assets/deep-q-learning.png" width=450px>

Now, our Q value, $Q(s, a)$ is calculated by passing in a state to the network. The output will be Q-values for each available action, with fully connected hidden layers.

<img src="assets/q-network.png" width=550px>


As I showed before, we can define our targets for training as $\hat{Q}(s,a) = r + \gamma \max{Q(s', a')}$. Then we update the weights by minimizing $(\hat{Q}(s,a) - Q(s,a))^2$. 

For this Cart-Pole game, we have four inputs, one for each value in the state, and two outputs, one for each action. To get $\hat{Q}$, we'll first choose an action, then simulate the game using that action. This will get us the next state, $s'$, and the reward. With that, we can calculate $\hat{Q}$ then pass it back into the $Q$ network to run the optimizer and update the weights.

Below is my implementation of the Q-network. I used two fully connected layers with ReLU activations. Two seems to be good enough, three might be better. Feel free to try it out.

In [13]:
# Data of the model
def model_input(state_size):
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    targetQs = tf.placeholder(tf.float32, [None], name='targetQs')
    reward = tf.placeholder(tf.float32, [], name='reward')
    return states, actions, targetQs, reward

In [14]:
# Generator/Controller: Generating/prediting the actions
def generator(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        #predictions = tf.nn.softmax(logits)

        # return actions logits
        return logits

In [15]:
# Discriminator/Dopamine: Reward function/planner/naviator/advisor/supervisor/cortical columns
def discriminator(states, actions, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # Fusion/merge states and actions/ SA/ SM
        x_fused = tf.concat(axis=1, values=[states, actions])
        
        # First fully connected layer
        h1 = tf.layers.dense(inputs=x_fused, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        #predictions = tf.nn.softmax(logits)

        # return rewards logits
        return logits

In [16]:
def model_loss(action_size, hidden_size, states, actions, targetQs, reward):
    # G
    actions_logits = generator(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    neg_log_prob_actions = tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels)
    g_loss = tf.reduce_mean(neg_log_prob_actions[:-1] * targetQs[1:])
    
    # D
    Qs_logits = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states)
    d_lossR = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qs_logits,
                                                                     labels=reward*tf.ones_like(Qs_logits)))
    d_lossQ = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qs_logits[:-1],
                                                                     labels=tf.reshape(tf.nn.sigmoid(targetQs[1:]),
                                                                                       shape=[-1, 1])))
    d_loss = d_lossR + d_lossQ

    return actions_logits, Qs_logits, g_loss, d_loss, d_lossR, d_lossQ

In [17]:
# Optimizating/training/learning G & D
def model_opt(g_loss, d_loss, learning_rate):
    """
    Get optimization operations in order
    :param g_loss: Generator loss Tensor for action prediction
    :param d_loss: Discriminator loss Tensor for reward prediction for generated/prob/logits action
    :param learning_rate: Learning Rate Placeholder
    :return: A tuple of (qfunction training, generator training, discriminator training)
    """
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        g_opt = tf.train.AdamOptimizer(learning_rate).minimize(g_loss, var_list=g_vars)
        d_opt = tf.train.AdamOptimizer(learning_rate).minimize(d_loss, var_list=d_vars)

    return g_opt, d_opt

In [18]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.targetQs, self.reward = model_input(state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.Qs_logits, self.g_loss, self.d_loss, self.d_lossR, self.d_lossQ = model_loss(
            action_size=action_size, hidden_size=hidden_size, # model init parameters
            states=self.states, actions=self.actions, # model input
            targetQs=self.targetQs, reward=self.reward) # model input
        
        # Update the model: backward pass and backprop
        self.g_opt, self.d_opt = model_opt(g_loss=self.g_loss, d_loss=self.d_loss, learning_rate=learning_rate)

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [19]:
print('state size:{}'.format(states.shape), 
      'actions:{}'.format(actions.shape)) 
print('action size:{}'.format(np.max(actions) - np.min(actions)+1))

state size:(1111, 4) actions:(1111,)
action size:2


In [20]:
# Training parameters
# Network parameters
state_size = 4               # number of units for the input state/observation -- simulation
action_size = 2              # number of units for the output actions -- simulation
hidden_size = 64             # number of units in each Q-network hidden layer -- simulation
learning_rate = 0.001          # learning rate for adam

In [21]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, hidden_size=hidden_size, state_size=state_size, learning_rate=learning_rate)

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [22]:
# import gym

# ## Create the Cart-Pole game environment
# env = gym.make('CartPole-v0')
# env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# # env = gym.make('MountainCar-v0')
# # env = gym.make('Pendulum-v0')
# # env = gym.make('Blackjack-v0')
# # env = gym.make('FrozenLake-v0')
# # env = gym.make('AirRaid-ram-v0')
# # env = gym.make('AirRaid-v0')
# # env = gym.make('BipedalWalker-v2')
# # env = gym.make('Copy-v0')
# # env = gym.make('CarRacing-v0')
# # env = gym.make('Ant-v2') #mujoco
# # env = gym.make('FetchPickAndPlace-v1') # mujoco required!

In [ ]:
from collections import deque
episodes_total_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window
saver = tf.train.Saver()
rewards_list, g_loss_list, d_loss_list = [], [], []
d_lossR_list, d_lossQ_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Training episodes/epochs
    for ep in range(111111):
        batch = [] # every data batch
        total_reward = 0
        state = env.reset() # env first state

        # Training steps/batches
        while True:
            action_logits, Q_logits = sess.run(fetches=[model.actions_logits, model.Qs_logits], 
                                               feed_dict={model.states: np.reshape(state, [1, -1])})
            action = np.argmax(action_logits)
            batch.append([state, action, Q_logits])
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done is True: # episode ended success/failure
                episodes_total_reward.append(total_reward) # stopping criteria
                rate = total_reward/ 500 # success is 500 points
                break

        # Training using batches
        #batch = memory.buffer
        states = np.array([each[0] for each in batch])
        actions = np.array([each[1] for each in batch])
        targetQs = np.array([each[2] for each in batch])
        g_loss, d_loss, d_lossR, d_lossQ, _, _ = sess.run([model.g_loss, model.d_loss, 
                                                           model.d_lossR, model.d_lossQ, 
                                                           model.g_opt, model.d_opt],
                                                          feed_dict = {model.states: states, 
                                                                       model.actions: actions,
                                                                       model.reward: rate,
                                                                       model.targetQs: targetQs.reshape([-1])})
        # Average 100 episode total reward
        # Print out
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episodes_total_reward)),
              'gloss:{:.4f}'.format(g_loss),
              'dloss:{:.4f}'.format(d_loss),
              'dlossR:{:.4f}'.format(d_lossR),
              'dlossQ:{:.4f}'.format(d_lossQ),
)
        # Ploting out
        rewards_list.append([ep, np.mean(episodes_total_reward)])
        g_loss_list.append([ep, g_loss])
        d_loss_list.append([ep, d_loss])
        d_lossR_list.append([ep, d_lossR])
        d_lossQ_list.append([ep, d_lossQ])
        # Break episode/epoch loop
        if np.mean(episodes_total_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:10.0000 gloss:0.1741 dloss:1.5524 dlossR:0.8816 dlossQ:0.6707
Episode:1 meanR:10.0000 gloss:0.1235 dloss:1.5145 dlossR:0.8354 dlossQ:0.6791
Episode:2 meanR:10.0000 gloss:0.0830 dloss:1.4828 dlossR:0.7980 dlossQ:0.6847
Episode:3 meanR:9.7500 gloss:0.0384 dloss:1.4333 dlossR:0.7426 dlossQ:0.6907
Episode:4 meanR:9.8000 gloss:0.0112 dloss:1.4060 dlossR:0.7136 dlossQ:0.6924
Episode:5 meanR:9.6667 gloss:-0.0167 dloss:1.3685 dlossR:0.6755 dlossQ:0.6929
Episode:6 meanR:9.5714 gloss:-0.0429 dloss:1.3352 dlossR:0.6436 dlossQ:0.6916
Episode:7 meanR:9.3750 gloss:-0.0701 dloss:1.3005 dlossR:0.6118 dlossQ:0.6887
Episode:8 meanR:9.5556 gloss:-0.0846 dloss:1.2514 dlossR:0.5699 dlossQ:0.6815
Episode:9 meanR:9.5000 gloss:-0.1164 dloss:1.2272 dlossR:0.5504 dlossQ:0.6768
Episode:10 meanR:9.4545 gloss:-0.1439 dloss:1.1887 dlossR:0.5206 dlossQ:0.6681
Episode:11 meanR:9.4167 gloss:-0.1722 dloss:1.1518 dlossR:0.4937 dlossQ:0.6581
Episode:12 meanR:9.3846 gloss:-0.2002 dloss:1.1282 dlossR:0.4779

Episode:114 meanR:18.5000 gloss:-2.2392 dloss:0.2747 dlossR:0.1118 dlossQ:0.1629
Episode:115 meanR:18.5600 gloss:-2.1951 dloss:0.3101 dlossR:0.1382 dlossQ:0.1719
Episode:116 meanR:18.6300 gloss:-2.1002 dloss:0.3350 dlossR:0.1459 dlossQ:0.1891
Episode:117 meanR:18.8500 gloss:-1.9836 dloss:0.4386 dlossR:0.2280 dlossQ:0.2106
Episode:118 meanR:19.0400 gloss:-2.0121 dloss:0.4261 dlossR:0.2221 dlossQ:0.2040
Episode:119 meanR:19.3300 gloss:-1.9838 dloss:0.4830 dlossR:0.2734 dlossQ:0.2096
Episode:120 meanR:19.8500 gloss:-1.9696 dloss:0.6158 dlossR:0.4042 dlossQ:0.2116
Episode:121 meanR:20.1600 gloss:-1.9721 dloss:0.5018 dlossR:0.2903 dlossQ:0.2114
Episode:122 meanR:20.5600 gloss:-1.9763 dloss:0.5466 dlossR:0.3362 dlossQ:0.2104
Episode:123 meanR:20.8800 gloss:-1.9733 dloss:0.5017 dlossR:0.2903 dlossQ:0.2114
Episode:124 meanR:21.1000 gloss:-2.0020 dloss:0.4452 dlossR:0.2394 dlossQ:0.2058
Episode:125 meanR:21.1100 gloss:-2.1468 dloss:0.2977 dlossR:0.1153 dlossQ:0.1824
Episode:126 meanR:21.2800 gl

Episode:220 meanR:40.4800 gloss:-1.9827 dloss:0.3491 dlossR:0.1292 dlossQ:0.2200
Episode:221 meanR:40.1700 gloss:-1.9831 dloss:0.3453 dlossR:0.1246 dlossQ:0.2207
Episode:222 meanR:39.8700 gloss:-1.7454 dloss:0.4530 dlossR:0.1801 dlossQ:0.2729
Episode:223 meanR:39.5700 gloss:-1.8136 dloss:0.3842 dlossR:0.1328 dlossQ:0.2514
Episode:224 meanR:39.9700 gloss:-1.5921 dloss:0.7343 dlossR:0.4276 dlossQ:0.3066
Episode:225 meanR:41.6600 gloss:-1.6303 dloss:1.2354 dlossR:0.9399 dlossQ:0.2955
Episode:226 meanR:42.8900 gloss:-1.6287 dloss:1.0845 dlossR:0.7888 dlossQ:0.2957
Episode:227 meanR:44.3100 gloss:-1.6029 dloss:1.1348 dlossR:0.8320 dlossQ:0.3028
Episode:228 meanR:45.5100 gloss:-1.5991 dloss:1.0843 dlossR:0.7808 dlossQ:0.3035
Episode:229 meanR:46.3800 gloss:-1.5445 dloss:0.9126 dlossR:0.5942 dlossQ:0.3184
Episode:230 meanR:47.1000 gloss:-1.5363 dloss:0.8478 dlossR:0.5272 dlossQ:0.3207
Episode:231 meanR:46.9200 gloss:-1.3987 dloss:0.5710 dlossR:0.2097 dlossQ:0.3613
Episode:232 meanR:46.9400 gl

Episode:322 meanR:32.6200 gloss:-2.0160 dloss:0.4037 dlossR:0.1990 dlossQ:0.2047
Episode:323 meanR:32.7700 gloss:-2.0073 dloss:0.4116 dlossR:0.2050 dlossQ:0.2066
Episode:324 meanR:32.6100 gloss:-1.9650 dloss:0.5896 dlossR:0.3750 dlossQ:0.2145
Episode:325 meanR:31.8500 gloss:-1.9854 dloss:0.8616 dlossR:0.6518 dlossQ:0.2098
Episode:326 meanR:30.7500 gloss:-2.0000 dloss:0.4812 dlossR:0.2740 dlossQ:0.2072
Episode:327 meanR:29.9100 gloss:-1.9310 dloss:0.7003 dlossR:0.4784 dlossQ:0.2219
Episode:328 meanR:28.8600 gloss:-1.9347 dloss:0.5217 dlossR:0.3002 dlossQ:0.2215
Episode:329 meanR:28.4100 gloss:-1.8813 dloss:0.6508 dlossR:0.4177 dlossQ:0.2331
Episode:330 meanR:28.1700 gloss:-1.8494 dloss:0.6870 dlossR:0.4466 dlossQ:0.2404
Episode:331 meanR:28.2700 gloss:-1.8773 dloss:0.4677 dlossR:0.2331 dlossQ:0.2346
Episode:332 meanR:28.4800 gloss:-1.8060 dloss:0.5989 dlossR:0.3480 dlossQ:0.2509
Episode:333 meanR:28.5600 gloss:-1.8200 dloss:0.5025 dlossR:0.2544 dlossQ:0.2480
Episode:334 meanR:28.7500 gl

Episode:424 meanR:49.4700 gloss:-1.3879 dloss:0.7323 dlossR:0.3673 dlossQ:0.3651
Episode:425 meanR:48.8300 gloss:-1.3771 dloss:0.6563 dlossR:0.2878 dlossQ:0.3684
Episode:426 meanR:48.9500 gloss:-1.3532 dloss:0.7042 dlossR:0.3283 dlossQ:0.3758
Episode:427 meanR:48.6300 gloss:-1.3402 dloss:0.6818 dlossR:0.3018 dlossQ:0.3799
Episode:428 meanR:48.7300 gloss:-1.3111 dloss:0.7300 dlossR:0.3413 dlossQ:0.3887
Episode:429 meanR:48.3500 gloss:-1.3512 dloss:0.6164 dlossR:0.2393 dlossQ:0.3771
Episode:430 meanR:47.8800 gloss:-1.3585 dloss:0.6020 dlossR:0.2269 dlossQ:0.3751
Episode:431 meanR:47.8100 gloss:-1.3747 dloss:0.5959 dlossR:0.2254 dlossQ:0.3705
Episode:432 meanR:47.5700 gloss:-1.3603 dloss:0.6213 dlossR:0.2464 dlossQ:0.3749
Episode:433 meanR:47.4600 gloss:-1.3742 dloss:0.5963 dlossR:0.2256 dlossQ:0.3707
Episode:434 meanR:47.3300 gloss:-1.3754 dloss:0.6115 dlossR:0.2411 dlossQ:0.3704
Episode:435 meanR:47.2300 gloss:-1.4895 dloss:0.5158 dlossR:0.1780 dlossQ:0.3377
Episode:436 meanR:46.7300 gl

...

Episode:3463 meanR:173.0000 gloss:0.1528 dloss:1.2994 dlossR:0.6263 dlossQ:0.6731
Episode:3464 meanR:173.3400 gloss:-0.3085 dloss:1.6197 dlossR:0.9433 dlossQ:0.6764
Episode:3465 meanR:173.2800 gloss:0.1412 dloss:1.2908 dlossR:0.6195 dlossQ:0.6713
Episode:3466 meanR:176.8400 gloss:0.1136 dloss:1.3058 dlossR:0.6179 dlossQ:0.6879
Episode:3467 meanR:181.0100 gloss:0.0297 dloss:1.3670 dlossR:0.6727 dlossQ:0.6944
Episode:3468 meanR:184.3500 gloss:0.2568 dloss:1.2843 dlossR:0.6191 dlossQ:0.6653
Episode:3469 meanR:188.9000 gloss:0.0136 dloss:1.3797 dlossR:0.6866 dlossQ:0.6931
Episode:3470 meanR:193.5800 gloss:0.1395 dloss:1.2863 dlossR:0.5984 dlossQ:0.6879
Episode:3471 meanR:198.2500 gloss:0.1391 dloss:1.2866 dlossR:0.5991 dlossQ:0.6874
Episode:3472 meanR:203.0400 gloss:0.2877 dloss:1.1799 dlossR:0.5112 dlossQ:0.6688
Episode:3473 meanR:207.8000 gloss:0.1879 dloss:1.2511 dlossR:0.5675 dlossQ:0.6836
Episode:3474 meanR:211.1300 gloss:0.3883 dloss:1.2716 dlossR:0.6246 dlossQ:0.6470
Episode:3475 me

Episode:3563 meanR:274.9500 gloss:-0.8886 dloss:1.1246 dlossR:0.6022 dlossQ:0.5224
Episode:3564 meanR:270.6500 gloss:-1.4743 dloss:0.8248 dlossR:0.4505 dlossQ:0.3744
Episode:3565 meanR:265.8900 gloss:-1.9372 dloss:0.5126 dlossR:0.2318 dlossQ:0.2809
Episode:3566 meanR:261.0100 gloss:-2.4881 dloss:0.3734 dlossR:0.1761 dlossQ:0.1973
Episode:3567 meanR:256.1500 gloss:-2.4947 dloss:0.4118 dlossR:0.2016 dlossQ:0.2102
Episode:3568 meanR:253.0000 gloss:-1.1812 dloss:0.9529 dlossR:0.4791 dlossQ:0.4738
Episode:3569 meanR:251.1100 gloss:0.0552 dloss:1.3891 dlossR:0.7461 dlossQ:0.6430
Episode:3570 meanR:246.3700 gloss:-0.7112 dloss:0.9476 dlossR:0.3719 dlossQ:0.5757
Episode:3571 meanR:241.4800 gloss:-1.0938 dloss:0.7284 dlossR:0.2657 dlossQ:0.4626
Episode:3572 meanR:236.5900 gloss:-1.0426 dloss:0.7639 dlossR:0.2859 dlossQ:0.4780
Episode:3573 meanR:231.6900 gloss:-1.0975 dloss:0.7474 dlossR:0.2869 dlossQ:0.4605
Episode:3574 meanR:228.2700 gloss:-1.2209 dloss:0.6712 dlossR:0.2452 dlossQ:0.4261
Episo

Episode:3663 meanR:135.1700 gloss:-0.1553 dloss:1.3316 dlossR:0.6529 dlossQ:0.6786
Episode:3664 meanR:136.1200 gloss:0.1331 dloss:1.4225 dlossR:0.7616 dlossQ:0.6609
Episode:3665 meanR:138.0800 gloss:0.1515 dloss:1.4068 dlossR:0.7479 dlossQ:0.6589
Episode:3666 meanR:140.7200 gloss:0.1986 dloss:1.3752 dlossR:0.7154 dlossQ:0.6598
Episode:3667 meanR:143.1400 gloss:-0.0291 dloss:1.3928 dlossR:0.7240 dlossQ:0.6688
Episode:3668 meanR:144.9200 gloss:0.1256 dloss:1.3868 dlossR:0.7112 dlossQ:0.6757
Episode:3669 meanR:144.5500 gloss:0.1246 dloss:1.3799 dlossR:0.7027 dlossQ:0.6772
Episode:3670 meanR:147.5900 gloss:0.1209 dloss:1.3605 dlossR:0.6855 dlossQ:0.6750
Episode:3671 meanR:151.4400 gloss:0.1410 dloss:1.3289 dlossR:0.6536 dlossQ:0.6753
Episode:3672 meanR:156.3300 gloss:0.0302 dloss:1.3671 dlossR:0.6759 dlossQ:0.6912
Episode:3673 meanR:161.2300 gloss:0.0318 dloss:1.3660 dlossR:0.6750 dlossQ:0.6909
Episode:3674 meanR:166.1000 gloss:-0.0621 dloss:1.4349 dlossR:0.7403 dlossQ:0.6946
Episode:3675 

Episode:3763 meanR:333.4200 gloss:0.2657 dloss:1.1966 dlossR:0.5244 dlossQ:0.6722
Episode:3764 meanR:336.7700 gloss:0.2110 dloss:1.2384 dlossR:0.5561 dlossQ:0.6822
Episode:3765 meanR:339.6300 gloss:0.2415 dloss:1.2175 dlossR:0.5358 dlossQ:0.6818
Episode:3766 meanR:340.9300 gloss:0.4790 dloss:1.1751 dlossR:0.5439 dlossQ:0.6312
Episode:3767 meanR:343.3700 gloss:0.1883 dloss:1.2555 dlossR:0.5679 dlossQ:0.6877
Episode:3768 meanR:345.8300 gloss:0.3758 dloss:1.1170 dlossR:0.4587 dlossQ:0.6582
Episode:3769 meanR:347.3800 gloss:0.3978 dloss:1.1834 dlossR:0.5336 dlossQ:0.6498
Episode:3770 meanR:345.9700 gloss:-0.1045 dloss:1.3712 dlossR:0.6942 dlossQ:0.6770
Episode:3771 meanR:343.5100 gloss:-0.1179 dloss:1.3548 dlossR:0.6884 dlossQ:0.6664
Episode:3772 meanR:339.3000 gloss:-0.1965 dloss:1.2947 dlossR:0.6276 dlossQ:0.6671
Episode:3773 meanR:335.1100 gloss:-0.1534 dloss:1.3178 dlossR:0.6530 dlossQ:0.6648
Episode:3774 meanR:330.9500 gloss:-0.1419 dloss:1.3253 dlossR:0.6652 dlossQ:0.6601
Episode:377

Episode:3865 meanR:229.3300 gloss:-1.2831 dloss:0.8020 dlossR:0.3853 dlossQ:0.4167
Episode:3866 meanR:225.7200 gloss:-1.3848 dloss:0.7414 dlossR:0.3522 dlossQ:0.3892
Episode:3867 meanR:221.2100 gloss:-1.4882 dloss:0.7213 dlossR:0.3548 dlossQ:0.3665
Episode:3868 meanR:216.4700 gloss:-1.7151 dloss:0.5512 dlossR:0.2384 dlossQ:0.3128
Episode:3869 meanR:212.3500 gloss:-2.0517 dloss:0.4324 dlossR:0.1913 dlossQ:0.2411
Episode:3870 meanR:210.5900 gloss:-2.2651 dloss:0.3893 dlossR:0.1789 dlossQ:0.2104
Episode:3871 meanR:209.2000 gloss:-2.6340 dloss:0.3290 dlossR:0.1594 dlossQ:0.1696
Episode:3872 meanR:208.5600 gloss:-1.6909 dloss:0.6305 dlossR:0.2559 dlossQ:0.3746
Episode:3873 meanR:210.8600 gloss:-0.8100 dloss:1.5761 dlossR:1.0209 dlossQ:0.5552
Episode:3874 meanR:213.2400 gloss:-0.7511 dloss:1.5797 dlossR:1.0059 dlossQ:0.5738
Episode:3875 meanR:216.0500 gloss:-0.7149 dloss:1.6156 dlossR:1.0340 dlossQ:0.5817
Episode:3876 meanR:220.1500 gloss:-0.6589 dloss:1.8406 dlossR:1.2450 dlossQ:0.5956
Epis

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(g_loss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('G losses')

In [ ]:
eps, arr = np.array(d_loss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [28]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(1):
    #while True:
        state = env.reset()
        total_reward = 0

        # Steps/batches
        #for _ in range(111111111111111111):
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: np.reshape(state, [1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                break
        # Print and break condition
        print('total_reward: {}'.format(total_reward))
        if total_reward == 500:
            break
                
# Closing the env
env.close()

/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Restoring parameters from checkpoints/model2.ckpt
total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.